In [1]:
from qubit_adapt import qubit_adapt_vqe
from qubit_pool import QubitPool
from molecule_factory_with_sparse import MoleculeFactory

qubitpool = QubitPool()
molecule_factory = MoleculeFactory()


In [2]:
molecule_symbol = 'H2'
# In qubit ADAPT-VQE normally we choose the generalized single and double excitations
type_of_generator = 'singlet_gsd'
transform = 'JW'
active = False

In [3]:
r, geometry, charge, spin, basis = molecule_factory.get_parameters(molecule_symbol)
print(" --------------------------------------------------------------------------")
print("Running in the non active case: ")
print("                     molecule symbol: %s " %(molecule_symbol))
print("                     molecule basis: %s " %(basis))
print("                     type of generator: %s " %(type_of_generator))
print("                     transform: %s " %(transform))
print(" --------------------------------------------------------------------------")

print(" --------------------------------------------------------------------------")
print("                                                          ")
print("                      Generate Hamiltonians and Properties from :")
print("                                                          ")
print(" --------------------------------------------------------------------------")
print("                                                          ")
hamiltonian, hamiltonian_sparse, hamiltonian_sp, hamiltonian_sp_sparse, n_elec, noons_full, orb_energies_full, info = molecule_factory.generate_hamiltonian(molecule_symbol,active=active, transform=transform)
nbqbits = len(orb_energies_full)
hf_init = molecule_factory.find_hf_init(hamiltonian, n_elec, noons_full, orb_energies_full)
reference_ket, hf_init_sp = molecule_factory.get_reference_ket(hf_init, nbqbits, transform)
print(" --------------------------------------------------------------------------")
print("                                                          ")
print("                      Generate Cluster OPS from :")
print("                                                          ")
print(" --------------------------------------------------------------------------")
print("                                                          ")

pool_size,cluster_ops, cluster_ops_sp, cluster_ops_sparse = molecule_factory.generate_cluster_ops(molecule_symbol, type_of_generator=type_of_generator, transform=transform, active=active)

print('Pool size: ', pool_size)
print('length of the cluster OP: ', len(cluster_ops))
print('length of the cluster OPS: ', len(cluster_ops_sp))
print('length of the cluster _sparse: ', len(cluster_ops_sp))

 --------------------------------------------------------------------------
Running in the non active case: 
                     molecule symbol: H2 
                     molecule basis: 6-31g 
                     type of generator: singlet_gsd 
                     transform: JW 
 --------------------------------------------------------------------------
 --------------------------------------------------------------------------
                                                          
                      Generate Hamiltonians and Properties from :
                                                          
 --------------------------------------------------------------------------
                                                          
Number of electrons =  2
Number of qubits before active space selection =  8
Orbital energies =  [-0.59279976  0.23579258  0.77956294  1.39138474]
Nuclear repulsion =  0.70556961456
Noons =  [1.9705947706867002, 0.024127481185769687, 0.005019126

/home/haidora7789/OpenVQE/python/cleaned_code/qubit_adapt/molecule_factory_with_sparse.py:382: UserWarning: This guess_init_state function is deprecated.
  n_elec, noons_full, orb_energies_full, hamiltonian.hpqrs


 --------------------------------------------------------------------------
                                                          
                      Generate Cluster OPS from :
                                                          
 --------------------------------------------------------------------------
                                                          
HF energy= -1.1265450345356909
MP2 energy= -1.1440347834365336
FCI energy= -1.1516885475166099
Pool size:  66
length of the cluster OP:  66
length of the cluster OPS:  66
length of the cluster _sparse:  66


In [ ]:
nbqbits = hamiltonian_sp.nbqbits
# user can just type the name of pool wanted: full, full_without_Z, reduced_without_Z, YXXX, XYXX,XXYX,XXXY,random, two, four, eight
# pure_with_symmetry

# for example here user can put radom (YXXX, XYXX, XXYX, XXXY):
pool_type = 'random'
qubit_pool =qubitpool.generate_pool(cluster_ops)
len_returned_pool, returned_pool = qubitpool.generate_pool_without_cluster(pool_type=pool_type, 
                                                                        nbqbits=nbqbits, 
                                                                        qubit_pool=qubit_pool,
                                                                        molecule_symbol=molecule_symbol)
# or user can type:
# pool_condition='full_without_Z'
# len_returned_pool, returned_pool = qubitpool.generate_hamiltonian_from_cluster(pool_condition, cluster_ops, nbqbits)
pool_mix = returned_pool
pool_pure = returned_pool
print("length of the pool",len(pool_mix))
iterations_sim, iterations_ana, result_sim, result_ana = qubit_adapt_vqe(hamiltonian_sp, hamiltonian_sp_sparse,
       reference_ket, nbqbits, pool_mix, hf_init_sp, info['FCI'],
        chosen_grad = 1,
        adapt_conver    = 'norm',
        adapt_thresh    = 1e-07,
        adapt_maxiter   = 29,
        tolerance_sim = 1e-09,
        method_sim = 'BFGS')
print("iterations",iterations_sim)    
print("results",result_sim)

The current pool is random
50
reference_energy from the simulator: -1.126545034535691
reference_energy from the analytical calculations: -1.1265450345356913
 --------------------------------------------------------------------------
                                                          
                      Start Qubit ADAPT-VQE algorithm:
                                                          
 --------------------------------------------------------------------------
                                                          
 ------------------------------------------------------
        The number of maximum gradients inserted in each iteration: 1
 ------------------------------------------------------


 --------------------------------------------------------------------------
                         Qubit ADAPT-VQE iteration:  0
 --------------------------------------------------------------------------


 ------------------------------------------------------
        St